In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Client Data

- **Feature Matrix (`X`)** `m x n`: Created using `np.random.randn(m, n)`, which generates values from a standard normal distribution (mean=0, variance=1).
- **True Coefficients** (weights `w`) and **Intercept** (bias term `b`): Randomly sampled from a normal distribution to define the linear relationship.
- **Target Vector** (`y = X @ w + b`): Computed as a linear combination of features and coefficients, plus Gaussian noise to simulate real-world data variability.

In [2]:
def generate_linear_regression_data(m: int, n: int, mean=0, std=1, noise_std=0.5):
    """
    Generates synthetic data for linear regression.

    Parameters:
    - m: Number of samples.
    - n: Number of features per sample.
    - noise_std (float): Standard deviation of the Gaussian noise added to y.

    Returns:
    - X (np.ndarray): Feature matrix of shape (m, n).
    - y (np.ndarray): Target vector of shape (m,).
    """

    # Generate feature matrix X from a standard normal distribution
    X = np.random.normal(mean, std, (m, n))

    # Generate true coefficients (weights) and intercept (bias)
    w = np.random.randn(n, 1)  # true coefficients
    b = np.random.randn()      # intercept term

    y = X.dot(w) + b

    # Add Gaussian noise to the target values
    y += np.random.normal(0, noise_std, (m, 1))

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)

    # Sort
    idx = np.argsort(X_train[:, 0])
    X_train = X_train[idx, :]
    y_train = y_train[idx]

    return X_train, X_test, y_train, y_test

## Weight Matrix

We need to generate a matrix `W` `n+1+1 x n+2`, the columns of which represent `n+2` (why `n+2` – to make `W` a **square** matrix) randomly generated weight vectors, each of which contains `n+1` elements w0, w1, w2, etc. and

the additional row of `1`'s will count for **the free term coefficient** of a gradient function $ \frac{\partial L(\textbf{w}, \;\textbf{x})}{\partial w_j} $, where $ j ∈ [0, .., n] $.

Also, to guarantee that `W` is **invertible**, we will make it [Diagonally Dominant Matrix](https://stackoverflow.com/questions/73426718/generating-invertible-matrices-in-numpy-tensorflow) over columns as


"A **strictly diagonally dominant matrix** (or an irreducibly diagonally dominant matrix) is **non-singular**."

In [ ]:
def generate_weight_matrix(n):
    W = np.random.rand(n+1, n+2).astype(np.float32)
    W = np.concatenate([W, np.ones((1, n+2))], axis=0)

    diag = np.sum(np.abs(W), axis=0) + 1
    np.fill_diagonal(W, diag)
    W[n+1, n+1] = 1         # the row of 1's was also affected, so reassigning a value of 1 again.

    return W

def client_calculate_gradients(W, X, y):
    batch_size = X.shape[0]

    # Add an intercept column
    X = np.hstack([np.ones((batch_size, 1)), X])

    # Remove the last row of one's from W
    W = W[:-1, :]

    # Make Y matrix out of n+2 copies of y to count for n+2 random sets of weights
    Y = np.hstack([y]*(n+2))

    # Calculate the gradient dL/dw
    L = X.T@(X @ W - Y)# * (1/batch_size)       #! Protocol_v1 is without normalizing (dividing by batch_size)

    return L

In [4]:
SEED = 43
np.random.seed(SEED)

In [5]:
n = 7
CLIENT_NUM = 10

DATA_SIZE = np.random.randint(1000, 10000, size=CLIENT_NUM)
MEAN = np.random.randint(0, 20, size=CLIENT_NUM)
STD = np.random.randint(5, 10, size=CLIENT_NUM)
NOISE_STD = np.array([np.random.uniform(0, 0.5*std) for std in STD])          # low
# NOISE_STD = np.array([np.random.uniform(0.5*std, std) for std in STD])      # medium
# NOISE_STD = np.array([np.random.uniform(std, 2*std) for std in STD])        # high

In [6]:
data = []       # [(X_train, X_test, y_train, y_test), ...]
for i in range(CLIENT_NUM):
    data.append(generate_linear_regression_data(DATA_SIZE[i], n, MEAN[i], STD[i], NOISE_STD[i]))

In [7]:
# Generate weight matrix W
W = generate_weight_matrix(n)
# Check its determinant
np.linalg.det(W)

np.float64(541423.3997807435)

In [8]:
# Compute gradients for each client (forward pass)
gradients = []
for i in range(CLIENT_NUM):
    X_train = data[i][0]
    y_train = data[i][2]
    gradients.append(client_calculate_gradients(W, X_train, y_train))

### 1. Sum the gradient update matrices `Lᵢ`

`Lᵢ` is the matrix received from `i`th client, also `n+1 x n+2`.

### 2. Find the matrix C

$ C \times W = L $  
$ ^{n+1 \times n+2} $ $ ^{n+1+1 \times n+2}$ $^{=}$ $ ^{n+1 \times n+2} $

$ C = L \times W^{-1} $  
$ ^{n+1 \times n+2} $ $^{=}$ $ ^{n+1 \times n+2} $ $ ^{n+2 \times n+2}$

### 3. Find the optimal set of weights

Find `w_opt` such that `C @ w_opt = 0`

In [9]:
L = np.sum(gradients, axis=0)

# Calculate w_opt by the proposed algorithm
C = L @ np.linalg.inv(W)
A = C[:, :-1]
b = C[:, -1] * -1
w_opt = np.linalg.solve(A, b)    # [w0, w1, w2, ...]

X_test = data[0][1]
y_test = data[0][3]
for i in range(1, CLIENT_NUM):
    X_test = np.concatenate((X_test, data[i][1]), axis=0)
    y_test = np.concatenate((y_test, data[i][3]), axis=0)

# Predict
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
y_pred = X_test @ w_opt

In [10]:
# Print the results
print("CLIENT_DATA_SIZE:", DATA_SIZE)
print("CLIENT_MEAN:", MEAN)
print("CLIENT_STD:", STD)
print("CLIENT_NOISE_STD:", NOISE_STD)
print()
print("Overall MSE:", mean_squared_error(y_test, y_pred))
print("Overall R²:", r2_score(y_test, y_pred))

CLIENT_DATA_SIZE: [4392 3303 8985 3064 9499 9849 6307 6534 7999 1379]
CLIENT_MEAN: [11  1 15  2 11  2  3  4  4 16]
CLIENT_STD: [9 5 9 5 7 9 5 6 7 5]
CLIENT_NOISE_STD: [1.73419611 2.38622031 2.00591262 1.67431163 0.28875017 4.03694361
 0.74500875 0.78691447 0.01795342 1.35800629]

Overall MSE: 895.9494963502965
Overall R²: 0.23150839863972417


# Simulation

In [11]:
results = []
for i in range(3):
    np.random.seed(SEED)

    n = 7
    CLIENT_NUM = 10

    DATA_SIZE = np.random.randint(1000, 10000, size=CLIENT_NUM)
    MEAN = np.random.randint(0, 20, size=CLIENT_NUM)
    STD = np.random.randint(5, 10, size=CLIENT_NUM)
    match i:
        case 0:
            NOISE_STD = np.array([np.random.uniform(0, 0.5*std) for std in STD])        # low
        case 1:
            NOISE_STD = np.array([np.random.uniform(0.5*std, std) for std in STD])      # medium
        case 2:
            NOISE_STD = np.array([np.random.uniform(std, 2*std) for std in STD])        # high

    data = []       # [(X_train, X_test, y_train, y_test), ...]
    for i in range(CLIENT_NUM):
        data.append(generate_linear_regression_data(DATA_SIZE[i], n, MEAN[i], STD[i], NOISE_STD[i]))

    # Generate weight matrix W
    W = generate_weight_matrix(n)

    # Compute gradients for each client (forward pass)
    gradients = []
    for i in range(CLIENT_NUM):
        X_train = data[i][0]
        y_train = data[i][2]
        gradients.append(client_calculate_gradients(W, X_train, y_train))

    L = np.sum(gradients, axis=0)

    # Calculate w_opt by the proposed algorithm
    C = L @ np.linalg.inv(W)
    A = C[:, :-1]
    b = C[:, -1] * -1
    w_opt = np.linalg.solve(A, b)    # [w0, w1, w2, ...]

    X_test = data[0][1]
    y_test = data[0][3]
    for i in range(1, CLIENT_NUM):
        X_test = np.concatenate((X_test, data[i][1]), axis=0)
        y_test = np.concatenate((y_test, data[i][3]), axis=0)

    # Predict
    X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
    y_pred = X_test @ w_opt

    # Append to results
    results.append((mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)))

# Print the results
for mse, r2 in results:
    print(mse, r2)
# low
# medium
# high

895.9494963502965 0.23150839863972417
920.7673872813183 0.22450718683000936
1008.3695127822564 0.20612919890339254
